<a href="https://colab.research.google.com/github/KartohaWhy/my_colab/blob/main/Copy_%22LLM_RAG_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговое домашнее задание курса

В этом задании Вы потренируетесь применять различные модели из HuggingFace для решения задач NLP, а также поработаете с LLM и углубитесь в особенности промптинга для взаимодействия с моделями.

## Часть 1: тренируемся применять предобученные модели для решения различных NLP-задач

Загрузим датасет cnn-dailymail с ежедневными новостями.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

In [ ]:
import os
for file in os.listdir(path):
    print("📄", file)

In [ ]:
inner_path = os.path.join(path, "cnn_dailymail")
for file in os.listdir(inner_path):
    print("📄", file)

In [ ]:
import pandas as pd

df = pd.read_csv(os.path.join(inner_path, "train.csv"))
print(df.columns)
print(df.head())

## Задача 1: суммаризация

Эту задачу мы решим за вас. Последовательность следующая:

- создаем pipeline для суммаризации, указываем название модели
- берем первую новость из нашего датасета (текст этой новости)
- применяем модель суммаризации к этой новости, причем хотим саммари длины от 30 до 130 токенов. Смотрим результат.

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
df['article'][0]

In [ ]:
summary = summarizer(df['article'][0], max_length=130, min_length=30, do_sample=False)
summary[0]['summary_text']

Задание: выведите в цикле для первых 10 новостей из датасета их текст и саммари.

In [ ]:
for i in range(10):

    article = df['article'][i]

    print(f"--- Обработка новости №{i+1} ---")

    if pd.isna(article) or not str(article).strip():
        print("! Статья пустая или отсутствует. Пропускаем.")
        print("-" * 80)
        continue

    try:
        # Генерация саммари
        summary = summarizer(
            article[:1024],
            max_length=130,
            min_length=30,
            do_sample=False
        )

        # Вывод результатов
        print("\nИсходный текст (первые 500 символов):")
        print(str(article)[:500])
        print("\nСаммари:")
        print(summary[0]['summary_text'])

    except Exception as e:
        print(f"! Произошла ошибка при обработке: {e}")

    print("-" * 80)

## Задача 2: классификация

Теперь решим задачу классификации, а именно, анализа тональности новостей. Повторите последовательность действий:
- создайте pipeline для анализа тональности ("sentiment-analysis"), укажите название модели "cardiffnlp/twitter-roberta-base-sentiment-latest"
- возьмите первую новость из нашего датасета (текст этой новости)
- применяем модель классификации к этой новости, выведите результат


In [ ]:
# ваш код здесь
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

In [ ]:
df['article'][0]

In [ ]:
sentiment = sentiment_pipeline(df['article'][0])
print(sentiment[0]['label'])
print(sentiment[0]['score'])

Теперь в цикле примените классификатор к первым 10 новостям. Что произошло?

In [ ]:
# ваш код здесь
for i in range(10):
    # Получение статьи
    article = df['article'][i]

    # Генерация тональности
    sentiment = sentiment_pipeline(article)

    # Вывод результатов
    print(f"\nНовость №{i+1}:")
    print("\nИсходный текст:")
    print(article[:500])
    print("\nТональность:")
    print(sentiment[0]['label'])
    print("\nВероятность:")
    print(sentiment[0]['score'])
    print("-" * 80)

Для длинных новостей сначала сделайте суммаризацию, а затем применяйте классификатор. Проделайте это упражнение в цикле для первых 10 новостей. Теперь все получилось?

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

MAX_INPUT_CHARS = 4096

for i in range(10):
    article = df['article'][i]
    print(f"--- Обработка новости №{i+1} ---")

    try:

        article_text = str(article).strip()
        safe_article_chunk = article_text[:MAX_INPUT_CHARS]
        # Модель сама обрежет длинный текст, но лучше подавать ей разумный кусок
        summary_result = summarizer(
            safe_article_chunk,
            max_length=150,
            min_length=40,
            do_sample=False
        )
        summary_text = summary_result[0]['summary_text']


        sentiment = sentiment_pipeline(summary_text)


        print("\nИсходный текст (первые 500 символов):")
        print(f"{str(article)[:500]}...")

        print("\n Краткое содержание (Саммари):")
        print(summary_text)

        print("\n Тональность (по саммари):")
        label = sentiment[0]['label']
        score = sentiment[0]['score']


        print(f"Результат: {label}")
        print(f"Уверенность: {score:.2%}") # Выводим в процентах

    except Exception as e:
        print(f"! Произошла непредвиденная ошибка при обработке: {e}")

    finally:
        print("-" * 80 + "\n")

## Задача 3: распознавание именованных сущностей (NER)

Наконец, решим задачу поиска именованных сущностей в текстах новостей (Named Entity Recognition, NER). Повторите последовательность действий:

- создайте pipeline для NER ("ner"), укажите название модели "dslim/bert-base-NER". Также в пайплайне укажите aggregation_strategy="simple" (это способ определения именованных сущностей для сущностей из нескольких слов)
- возьмите первую новость из нашего датасета (текст этой новости)
- примените модель к этой новости, выведите результат

In [ ]:
# ваш код здесь
ner_pipeline = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)

In [ ]:
df['article'][0]

In [ ]:
ner = ner_pipeline(df['article'][0])
for entity in ner:
        print(
            f"Сущность: {entity['word']}, "
            f"Тип: {entity['entity_group']}, "
            f"Уверенность: {entity['score']:.2%}"
        )

Примените модель к первым 10 новостям из датасета. Если возникнут проблемы - попробуйте их исправить.

In [ ]:
# ваш код здесь

for i in range(10):
    try:
        # Получение текста новости
        article = df['article'][i]

        # Проверка на пустые значения
        if pd.isna(article):
            print(f"\nНовость №{i+1}: отсутствует текст")
            continue

        # Обрезаем текст, если он слишком длинный
        safe_article = str(article)[:4096].strip()

        # Извлечение сущностей
        entities = ner_pipeline(safe_article)

        # Вывод результатов
        print(f"\n--- Новость №{i+1} ---")
        print(f"Первые 500 символов текста:\n{str(article)[:500]}...")
        print("\nНайденные сущности:")

        if not entities:
            print("Сущности не найдены")
            continue

        for entity in entities:
            print(
                f"  Сущность: {entity['word']}, "
                f"Тип: {entity['entity_group']}, "
                f"Уверенность: {entity['score']:.2%}"
            )

    except Exception as e:
        print(f"\nОшибка при обработке новости №{i+1}: {str(e)}")

    finally:
        print("-" * 80)

## Часть 2: применяем большие генеративные модели

Поработаем с моделью [OpenChat](https://huggingface.co/openchat/openchat-3.5-0106).

Загрузите модель (запустите ячейки ниже).

In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

In [ ]:
import transformers

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'openchat/openchat-3.5-0106'

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
)

### Сколько гигабайт GPU заняла модель?

In [ ]:
# ваш код или проверка здесь

# Функция для подсчёта памяти
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def get_model_size_gb(model):
    total_params = count_parameters(model)
    total_bytes = total_params * 2
    return total_bytes / (1024 ** 3)  # Конвертация в гигабайты

# Подсчёт памяти
model_size_gb = get_model_size_gb(model)
print(f"Размер модели на GPU: {model_size_gb:.2f} ГБ")

# Дополнительно можно проверить использование памяти GPU
print("\nИспользование памяти GPU:")
print(f"Общая память GPU: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} ГБ")
print(f"Занято памяти GPU: {torch.cuda.memory_allocated() / (1024**3):.2f} ГБ")
print(f"Максимально использовалось памяти GPU: {torch.cuda.max_memory_allocated() / (1024**3):.2f} ГБ")

### Задание: с помощью OpenChat попробуйте решить все три задачи выше.

Решим первую задачу (анализ тональности) за вас. Мы попытались подобрать полезную инструкцию для модели.

Очень важно проверить, что среда выполнения в google colab - это GPU! Иначе генерация будет идти ооооочень медленно.

In [ ]:
prompt = "Classify the following article as positive, neutral or negative: " + df['article'][0]
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=50, # максимальная длина ответа
    temperature=0.7, # можно менять этот параметр для настройки вариативности/точности результата
    top_p=0.9, # можно менять этот параметр для настройки вариативности/точности результата
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:] # выводим на экран только сгенерированный текст, без исходного
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("📄 Ответ модели:")
print(response.strip())

Получилось не очень :( Как будто модель не поняла, чего мы от нее хотим.

Мы можем дать модели примеры ответов в удобном для нее шаблоне. Попробуем!

In [ ]:
prompt = """
GPT4 Correct Assistant:
You are a helpful assistant that classifies the sentiment of news headlines as Positive, Neutral, or Negative.
Classify the sentiment of each news item as Positive, Neutral, or Negative.

GPT4 Correct User:
Question: Apple's new iPhone sales break records in first weekend.
GPT4 Correct Assistant:
Correct Answer: POSITIVE <|end_of_turn|>

GPT4 Correct User:
Question: The city council met Tuesday to discuss zoning regulations.
GPT4 Correct Assistant:
Correct Answer: NEUTRAL <|end_of_turn|>

GPT4 Correct User:
Question: Severe floods displace thousands in southern regions.
GPT4 Correct Assistant:
Correct Answer: NEGATIVE <|end_of_turn|>

GPT4 Correct User: Question: """ + df['article'][0] + """
GPT4 Correct Assistant:
""".strip()

batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=50, # максимальная длина ответа
    temperature=0.7, # можно менять этот параметр для настройки вариативности/точности результата
    top_p=0.9, # можно менять этот параметр для настройки вариативности/точности результата
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:] # выводим на экран только сгенерированный текст, без исходного
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("📄 Ответ модели:")
print(response.strip())

Ваша очередь!

- Подберите промпт (текстовый запрос) для решения задачи суммаризации и запустите генерацию для первой новости из датасета
- Подберите промпт для решения задачи NER и запустите генерацию для первой новости из датасета

Оцените визуально результат и обязательно напишите текстом выводы:
- Хорошо ли справилась LLM?
- LLM справилась лучше, чем специализированные модели из части 1 или хуже?

In [ ]:
# ваш код здесь
# Промпт для суммаризации
summarization_prompt = f"""
GPT4 Summarizer:
You are an expert news summarizer. Create a concise summary of the news article in 3-4 sentences, preserving the main ideas and key facts.

Article: {df['article'][0]}

Summary:
"""

# Подготовка и генерация
batch = tokenizer(summarization_prompt, return_tensors='pt', return_token_type_ids=False).to(device)


In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=150,  # увеличено для более длинного ответа
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:]
summary = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n📊 Сгенерированная суммаризация:")
print(summary.strip())

In [ ]:
# Промпт для NER
ner_prompt = f"""
Named Entity Recognizer:
Extract all named entities from the text and classify them into categories:
- PERSON (люди)
- ORG (организации)
- LOC (места)
- DATE (даты)
- MONEY (денежные суммы)
- EVENT (события)

Format your response as a list of entities with their types:

Text: {df['article'][0]}

Entities:
"""

# Подготовка и генерация
batch = tokenizer(ner_prompt, return_tensors='pt', return_token_type_ids=False).to(device)

In [ ]:
output_tokens = model.generate(
    **batch,
    max_new_tokens=200,  # увеличено для более подробного ответа
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

generated_tokens = output_tokens[0][batch["input_ids"].shape[1]:]
entities = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("\n🔍 Извлеченные сущности:")
print(entities.strip())

# Выводы

LLM справилась с задачей суммаризации, но в отличии от специализированных моделей, добавила всё-таки лишнюю информацию. Она хуже концентрируется на основных фактах, добавляя лишние детали

Базовая суммаризация за счет моделей справилась всё-таки лучше, чем LLM.

С задачей NER LLM также справилась хуже моделей. Во-первых, она нашла меньше корректных именованных сущностей. Ошибки в определении нашлись в обоих случаях. Можно сказать, что LLM нашла те сущности, в которых у специализированных моделей уверенность около 99%. А сущности с уверенностью, например, 90% LLM найти уже не смогла

## Часть 3: творческая

Попробуйте [собрать RAG-систему](https://colab.research.google.com/drive/196TKVfLWesbrF7f4KNHMGZNJC3hfg5IK?usp=sharing), как мы делали это на вебинаре, для модели OpenChat (можно использовать код по ссылке, только внимательно смотрите, все ли параметры актуальны именно для этой модели).

В качестве данных давайте возьмем [rag-mini-bioasq](https://huggingface.co/datasets/enelpol/rag-mini-bioasq), это структурно схожый датасет с тем, что был в семинаре, но теперь он по био-медицинской тематике. Это сабдатасет с вокршопа [the BioASQ Challenge](https://www.bioasq.org/), то есть вы решаете задачу самого актуального соревнования в сфере био-медицинского NLP!

Основная сложность, с которой вы сталкнетесь, как оценить получившуюся RAG-систему. Авторы соревнования делают это [так](http://participants-area.bioasq.org/Tasks/b/eval_meas_2022/): они разбивают валидацию на две части - оценка retrievel и оценка сгенерированного ответа. Давайте сделаем так же, но упростим нашу систему.

Оба способа оценки вы реализуете самостоятельно. Для части с answer давайте отфильтруем вопросы формата yes/no и просто посчитаем F1 по двум классам. А для части с retrievel используем метрику Hit Rate@k - попал ли хоть один релевантный документ в первые *k*? Форма будет следующая для одного вопроса *i*:

Hit Rate_i\@k

  $$
    \text{success}_i =
      \begin{cases}
        1, & \text{если } |\,\text{gold}_i \cap \text{retrieved}_i^{[:k]}| > 0 \\
        0, & \text{иначе}
      \end{cases}
  $$

Затем агрегируем по всем *N* вопросам:

$$
  \text{Hit Rate@k} = \frac{1}{N}\sum_{i=1}^{N}\text{success}_i.
$$

Другими словами, мы просто для каждого вопроса считаем, попал ли он в топ_k или нет, а потом усредняем по всем вопросам. Давайте в качестве k возьмем 3, то есть будем считать Hit Rate@3.

*Пример*

| Вопрос | Релевантных доков (gold) | Что вернула система (топ-3) | Hit Rate\@3   
| ------ | ------------------------ | --------------------------- | ------------
| Q1     | {A, B}                   | A, C, D                     | 1 (A найден)
| Q2     | {E}                      | F, G, H                     | 0            
| Q3     | {J, K, L}                | K, L, A                     | 1 (A найден)      

Средние значения: *Hit Rate\@3* = (1 + 0 + 1)/3 = 0.67

Осталось понять, где взять золотой ответ и что делать, если вы используете разбиение на чанки. 1) в датасете есть relevant_passage_ids - это и есть золотые (golden) ответы. 2) при разбиении на чанки сохраняйте информацию о том, к какого relevant_passage_ids он относится. Например, вот так: metadata={"id": rec["id"], "chunk": i}.

Если вы вдруг в процессе реализации задания столкнетесь со сложностями, не бойтесь подглядывать в решения других датасаентистов. Например, [kaggle](https://www.kaggle.com/) - это кладезь отличных готовых решений различных задач, и RAG в том числе. Можно, допустим, заглянуть в [ноутбук](https://www.kaggle.com/code/erwanversmee/rag-for-mini-bioasq/notebook) к Erwan Versmee (если найдете решение полезным, поставьте лайк - человеку будет приятно). Каждый код индивидуален, и чем больше вы увидите разных вариаций, тем сильнее станете как специалист. Удачи в решении задания!

# Pips

In [ ]:
# ваш код здесь

!pip install -q --upgrade pip
!pip uninstall -y fastai torch torchvision torchaudio gcsfs fsspec

In [ ]:
!pip install --upgrade transformers torch bitsandbytes

In [ ]:
!pip install -q torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 \
    --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install -q fsspec==2025.3.2 gcsfs==2025.3.2

In [ ]:
!pip install -q \
    bitsandbytes==0.46.0 \
    transformers==4.52.4 \
    accelerate==1.8.1 \
    datasets==3.6.0 \
    sentence-transformers==4.1.0 \
    chromadb==1.0.13 \
    langchain==0.3.26 \
    langchain-community==0.3.26 \
    langchain-huggingface \
    tqdm==4.67.1

In [ ]:
!pip uninstall transformers bitsandbytes
!pip install transformers==4.32.0 bitsandbytes
!pip install langchain_community chromadb sentence-transformers

# BitsAndBytes

In [ ]:
from torch import cuda, bfloat16
import torch, transformers
import torchvision
from torch.optim.lr_scheduler import LRScheduler

In [ ]:
# from torch import cuda, bfloat16
# import torch, transformers
# import torchvision
# from torch.optim.lr_scheduler import LRScheduler
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline


torchvision.disable_beta_transforms_warning()
DEVICE = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"

In [ ]:
import sys
print(transformers.__version__, sys.executable)

In [ ]:
import bitsandbytes as bnb, importlib, os
print(bnb.__version__)
!ldconfig -p | grep cusparse | head -n 3

# LLM

In [ ]:
model_name = 'openchat/openchat-3.5-0106'

# Квантуем в 4 бита, чтобы поместилось в VRAM 6–8 ГБ
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                          # включить 4-битное квантование
    bnb_4bit_quant_type="nf4",                  # тип квантования: "nf4" (Normalized Float 4) или "fp4"
    bnb_4bit_use_double_quant=True,             # включить двойное квантование (дополнительная компрессия)
    bnb_4bit_compute_dtype=torch.float16        # тип данных для вычислений (например, bfloat16 (недоступен на T4), float16)
)


print("Загружаем модель …")
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    # trust_remote_code=True,
)

# tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id


# Данные

In [ ]:
from pathlib import Path

from datasets import load_dataset
from langchain.docstore.document import Document

from langchain.document_loaders import TextLoader  # загружает текстовые файлы и превращает их в объекты Document для LangChain.
from langchain.text_splitter import RecursiveCharacterTextSplitter  # рекурсивно разбивает длинный текст на более мелкие фрагменты (chunks).
from langchain_huggingface import HuggingFaceEmbeddings  # оборачивает модели из HuggingFace для получения эмбеддингов текста.
from langchain.vectorstores import Chroma  # векторное хранилище Chroma: сохраняет и ищет эмбеддинги.

from langchain_huggingface import HuggingFacePipeline  # использует HuggingFace Transformers pipeline как LLM-модуль в LangChain.
from langchain.chains import RetrievalQA  # готовая цепочка «поиск + генерация ответа» (Retrieval-augmented QA).

In [ ]:
bio_ds = load_dataset("rag-datasets/rag-mini-bioasq", name="text-corpus", split="passages")

# Создание документов
corpus_docs = [
    Document(page_content=rec["passage"], metadata={"id": rec["id"]})
    for rec in bio_ds
]

print(f"Загружено документов: {len(corpus_docs)}")
print("Пример документа:\n", corpus_docs[0].page_content[:200], "...")

In [ ]:
bio_ds

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=50
)

# docs = splitter.split_documents(corpus_docs)
# print("Чанков:", len(docs))

docs = []
for doc in corpus_docs:
    chunks = splitter.split_documents([doc])
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk"] = i
        docs.append(chunk)

print("Чанков:", len(docs))

# Создание векторной базы

In [ ]:
persist_directory = "chroma_ragmini"
if os.path.exists(persist_directory):
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
else:
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2",
        model_kwargs={"device": "cuda"}
    )
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vectordb.persist()

# Сборка цепочки

In [ ]:
import numpy as np

In [ ]:
import ast

def evaluate_retrieval(retriever, val_ds, k=3):
    # Загружаем только первые 10 примеров
    val_ds = load_dataset("rag-datasets/rag-mini-bioasq",
                         name="question-answer-passages",
                         split="test").select(range(10))

    hit_rates = []
    results = []  # Для хранения детальной информации

    for example in val_ds:
        question = example["question"]

        try:
            # Преобразуем строку в список чисел
            relevant_ids_str = example["relevant_passage_ids"]
            gold_ids = set(ast.literal_eval(relevant_ids_str))

            # Получаем топ-k документов
            docs = retriever.get_relevant_documents(question)[:k]
            retrieved_ids = set(doc.metadata["id"] for doc in docs)

            # Проверяем пересечение с золотыми id
            hit = 1 if gold_ids.intersection(retrieved_ids) else 0
            hit_rates.append(hit)

            # Сохраняем детальную информацию
            results.append({
                "question": question,
                "gold_ids": gold_ids,
                "retrieved_ids": retrieved_ids,
                "hit": hit
            })

        except Exception as e:
            print(f"Ошибка при обработке вопроса '{question}': {e}")
            hit_rates.append(0)
            results.append({
                "question": question,
                "error": str(e)
            })

    # Выводим детальную информацию
    print("\nПодробные результаты:")
    for res in results:
        print(f"\nВопрос: {res['question']}")
        print(f"Золотые ID: {res['gold_ids']}")
        print(f"Найденные ID: {res['retrieved_ids']}")
        print(f"Hit: {'Да' if res.get('hit', 0) else 'Нет'}")

    return np.mean(hit_rates) if hit_rates else 0.0

In [ ]:

def evaluate_answers(qa_chain, val_ds):
    val_ds = load_dataset("rag-datasets/rag-mini-bioasq",
                         name="question-answer-passages",
                         split="test").select(range(10))

    true_labels = []
    predicted_labels = []
    results = []

    print("\nРезультаты оценки:")

    for example in val_ds:
        question = example["question"]
        gold_answer = example["answer"].strip().lower()

        # Определяем тип вопроса
        is_yes_no = gold_answer in ["yes", "no"]

        try:
            response = qa_chain.invoke({"query": question})["result"].strip().lower()

            if is_yes_no:
                true_label = 1 if gold_answer == "yes" else 0
                predicted = 1 if response == "yes" else 0

                true_labels.append(true_label)
                predicted_labels.append(predicted)

                results.append({
                    "question": question,
                    "gold_answer": gold_answer,
                    "model_response": response,
                    "correct": true_label == predicted
                })

                print(f"\nВопрос: {question}")
                print(f"Золотой ответ: {gold_answer}")
                print(f"Ответ модели: {response}")
                print(f"Верно: {'Да' if true_label == predicted else 'Нет'}")

            else:
                # print(f"\nВопрос: {question}")
                # print(f"Золотой ответ: {gold_answer}")
                # print(f"Ответ модели: {response}")
                print("Это не yes/no вопрос, оценка не производится")

        except Exception as e:
            print(f"Ошибка при обработке вопроса '{question}': {e}")

    if true_labels:
        f1 = f1_score(true_labels, predicted_labels)
        print(f"\nF1 score для yes/no вопросов: {f1:.4f}")
    else:
        print("\nНет yes/no вопросов в выборке")

    return f1

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from transformers import GenerationConfig
from langchain import PromptTemplate


In [ ]:
gen_cfg = GenerationConfig(
    max_length=2048,
    max_new_tokens=128,
    # temperature=0.1,
    # do_sample=False,
    # top_p=0.9
)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto"
    ),
    model_kwargs={
        "max_new_tokens": 128,
        "temperature": 0.1,
        "do_sample": False
    }
)

In [ ]:
print(val_ds.column_names)

In [ ]:
# Обновленный промпт
# template = """
# Answer the question based on the provided context. Follow these rules strictly:

# 1. For yes/no questions:
#    - Respond ONLY with "Yes" or "No"
#    - No additional explanations

# 2. For open-ended questions:
#    - Provide a concise answer (1-2 sentences)
#    - Use only information from the context
#    - Keep the answer relevant and to the point

# 3. DO NOT include:
#    - Any instructions
#    - Role descriptions
#    - Extra text

# Context:
# {context}

# Question: {question}

# Answer:
# """
# prompt = PromptTemplate(
#     template=template,
#     input_variables=["context", "question"]
# )


In [ ]:
# Максимально упрощенный промпт
template = """
Answer ONLY the question with a direct response.

Question: {question}
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)

In [ ]:
val_ds = load_dataset("rag-datasets/rag-mini-bioasq", name="question-answer-passages", split="test").select(range(10))

# test_samples = val_ds.select(range(10))

In [ ]:
for i, example in enumerate(val_ds):
        if i >= 10:  # Ограничение до 10 примеров
            break

        question = example["question"]
        answer = example["answer"].lower()

        print(f"\nВопрос {i+1}: {question}")
        print(f"Ответ: {answer}")

        # Проверяем, является ли вопрос yes/no
        if answer in ["yes", "no"]:
            print("Тип вопроса: Yes/No")
        else:
            print("Тип вопроса: Открытый")

In [ ]:
from langchain.chains import LLMChain

# Создаем цепочку
chain = LLMChain(llm=llm, prompt=prompt)

def evaluate_single_answer(chain, question, gold_answer):
    try:
        # Получаем полный ответ модели
        full_response = chain.run(question)

        print(f"\nПОЛНЫЙ ОТВЕТ МОДЕЛИ: {full_response}")

        # Очищаем ответ от лишних символов
        cleaned_response = full_response.strip().lower().split()[0] if full_response else ""

        # Определяем тип вопроса
        is_yes_no = gold_answer.lower() in ["yes", "no"]

        if is_yes_no:
            true_label = 1 if gold_answer.lower() == "yes" else 0
            predicted = 1 if cleaned_response == "yes" else 0

            print(f"\nВопрос: {question}")
            print(f"Золотой ответ: {gold_answer}")
            print(f"Очищенный ответ модели: {cleaned_response}")
            print(f"Верно: {'Да' if true_label == predicted else 'Нет'}")

            return true_label, predicted
        else:
            print("Это не yes/no вопрос")
            return None, None

    except Exception as e:
        print(f"Ошибка при обработке вопроса: {e}")
        return None, None

# Тестируем на конкретном примере
question = "Is the protein Papilin secreted?"
gold_answer = "Yes"

true_label, predicted = evaluate_single_answer(chain, question, gold_answer)

In [ ]:
hit_rate = evaluate_retrieval(retriever, val_ds, k=3)
print(f"Hit Rate@3: {hit_rate:.4f}")

answer_f1 = evaluate_answers(qa_chain, val_ds)
print(f"F1 score для yes/no вопросов: {answer_f1:.4f}")

# Результаты

Мне удалось достигнуть метрики hit rate@k = 0.7

ID документов действительно верно находятся во всех правильных ответах.

Но, к сожалению, мне не удалось подстроить промпт и функцию для оценку yes/no ответов так, чтобы модель считывала их как подобные вопросы. Она всегда добавляла к ответам лишний контекст, хоть и сами ответы  yes/no в дальнейшем совпадали

Поэтому оценить f1-score можно было только вручную, что я, к сожалению, тоже забыл сделать перед изменениями (а времени на gpu для перезапуска оставалось немного). Но по моей памяти все ответы, которые я видел(5-10 штук), совпадали.